In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd "/content/drive/My Drive/CSE 244/abc"

/content/drive/My Drive/CSE 244/abc


In [0]:
import nltk
nltk.download('punkt')
import os
from argparse import Namespace
import json
from tqdm.autonotebook import tqdm
import numpy as np
import string
import io
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import sys
import numpy as np
from matplotlib import pyplot
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from sklearn.preprocessing import OneHotEncoder
#from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from termcolor import colored
from collections import Counter
try:
  import seqeval.metrics
except:
  !pip install seqeval
  import seqeval.metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=8e82e4d6ec9dd1dd81323c7cc6b9cf7114750649b49b3014125d1b253dc34510
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
os.getcwd()

'/content/drive/My Drive/CSE 244/abc'

In [0]:
# variables defined in the arguments to parse
args = Namespace(
    dev_path = os.getcwd() + '/dataset/dev-v1.1.json',
    train_path = os.getcwd() + '/dataset/train-v1.1.json',
    glove_dir = os.path.join(os.getcwd(), "glove.6B"),
    # glove_dir = '/content/drive/My Drive/Study/Winter 2020/CSE 244/Project/jojonki_BIDAF/dataset/',
    glove_corpus = '6B',
    glove_vec_size = 50,
    num_epochs = 50,
    batch_size =64
)

In [0]:
# Preprocessing 

# Class to store Question and corresponding list of Answers
class QueAns:
    def __init__(self, question, answers):
        self.question = question
        self.answers = answers

# Class to store information of Answer : Answer Text, Answer Start, Answer End
class Answer:
    def __init__(self, answer_text, answer_start, answer_end):
        self.answer_text = answer_text
        self.answer_start = answer_start
        self.answer_end = answer_end

# Method to tokenize the string
def word_tokenize(str):
    return nltk.word_tokenize(str.lower())

# Method to return the contexts and QueAns object in dictionary
def get_context_qa(data_path):
    source_data = json.load(open(data_path, 'r'))     
    contexts = dict()
    para_que_ans = dict()

    context_id = 0
    for data_index, data in enumerate(tqdm(source_data['data'])):  
        for para_index, paras in enumerate(data['paragraphs']):
            
            context = paras['context']
            context = context.replace("''", '"')            
            contexts[context_id] = word_tokenize(context)
            
            que_ans = list()
            for qa in paras['qas']:

                question = word_tokenize(qa['question'])
                
                answers = []
                for answer in qa['answers']:                    
                    answer_text = word_tokenize(answer['text'])
                    answer_start, answer_stop = getWordIndices(contexts[context_id], answer)
                    a1 = Answer(answer_text, answer_start, answer_stop)
                    answers.append(a1)

                q_a = QueAns(question, answers)
                que_ans.append(q_a)

            para_que_ans[context_id] = que_ans
            context_id += 1

    return contexts, para_que_ans

# Method to create the vocabulary, glove embeddings and weight matrix 
def get_word2vec():
    dev_data = json.load(open(args.dev_path, 'r'))
    train_data = json.load(open(args.train_path, 'r'))

    vocab = dict()
    rev_vocab = dict()
    word2vec = dict()
    
    vocab['<PAD>'] = 0
    rev_vocab[0] = '<PAD>'
    
# Creating Vocabulary from Dev data here
    vocab_count = 1
    print('Creating Vocabulary...')
    print('Processing Dev data...')
    for data in tqdm(dev_data['data']):
        for paras in data['paragraphs']:
            context = paras['context']
            context = context.replace("''", '"')
            context = context.replace("``", '"')
            for word in word_tokenize(context):
                if word not in vocab:
                    vocab[word] = vocab_count
                    rev_vocab[vocab_count] = word
                    vocab_count += 1
            for qa in paras['qas']:
                for word in word_tokenize(qa['question']):
                    if word not in vocab:
                        vocab[word] = vocab_count
                        rev_vocab[vocab_count] = word
                        vocab_count += 1
                for answer in qa['answers']:
                    for word in word_tokenize(answer['text']):
                        if word not in vocab:
                            vocab[word] = vocab_count
                            rev_vocab[vocab_count] = word
                            vocab_count += 1

# Creating Vocabulary from training data
    print('Processing train data...')
    for data in tqdm(train_data['data']):
        for paras in data['paragraphs']:
            context = paras['context']
            context = context.replace("''", '"')
            for word in word_tokenize(context):
                if word not in vocab:
                    vocab[word] = vocab_count
                    rev_vocab[vocab_count] = word
                    vocab_count += 1
                    
            for qa in paras['qas']:
                for word in word_tokenize(qa['question']):
                    if word not in vocab:
                        vocab[word] = vocab_count
                        rev_vocab[vocab_count] = word
                        vocab_count += 1
                    
                for answer in qa['answers']:
                    for word in word_tokenize(answer['text']):
                        if word not in vocab:
                            vocab[word] = vocab_count
                            rev_vocab[vocab_count] = word
                            vocab_count += 1

    print('Vocabulary created!')
    print('Now creating word2vec dictionary...')

# Creating the word2vec with the GloVe embeddings and weight matrix
    glove_path = os.path.join(args.glove_dir, "glove.{}.{}d.txt".format(args.glove_corpus, args.glove_vec_size))
    print('Glove Path: ', glove_path)

    sizes = {'6B': int(4e5), '42B': int(1.9e6), '840B': int(2.2e6), '2B': int(1.2e6)}
    total = sizes[args.glove_corpus]
    print('Glove Corpus Size: ', total )

    with open(glove_path, 'r', encoding='utf-8') as fh:
        for line in tqdm(fh, total=total):
            array = line.lstrip().rstrip().split(" ")
            word = array[0]
            vector = list(map(float, array[1:]))

            if word.lower() in vocab:
                word2vec[word] = vector
    
    print('Glove word2vec dictionary created!')
    print('Generating weight matrix...')

    weights_matrix = np.zeros((len(vocab), int(args.glove_vec_size)))
    for i, (key, value) in enumerate(tqdm(vocab.items())):
        try:
            weights_matrix[i] = word2vec[value]
        except KeyError:
            weights_matrix[i] = np.random.normal(scale = 0.6, size = args.glove_vec_size)
    
    print('Weight matrix generated!')
    return vocab, rev_vocab, word2vec, weights_matrix

# Method to get the contexts, QueAns dictionaries, vocabulary, GloVe embeddings and weight matrix
def preprocess_data():
    print('Getting paragraphs, questions and answers...')
    
    paras_dev, para_qa_dev = get_context_qa(args.dev_path)      # eventually will be used as test set
    paras_train, para_qa_train = get_context_qa(args.train_path)  # will have to split to get the val set
    vocab, rev_vocab, word2vec, weights_matrix = get_word2vec()
    
    print('Returning paragraphs, questions and answers...')
    return paras_dev, para_qa_dev, paras_train, para_qa_train, vocab, rev_vocab, word2vec, weights_matrix

# Method to return indices in a list of words from vocabulary for the given text
def get_indices(text, vocab):
    text_indices = list()
    for word in text:
        text_indices.append(vocab.get(word))

    return text_indices

# Method to return the max_length of all the contexts
# This will be useful in padding the contexts
def get_max_length_paras(paras_train, paras_dev):
    length_train = len(max(paras_train.values(), key = len))
    length_dev = len(max(paras_dev.values(), key = len))

    if length_train > length_dev:
        return length_train
    else:
        return length_dev

# Method to return the max_length of all the queries
# This will be useful in padding the queries
def get_max_length_questions(para_qa_train, para_qa_dev):
    que_length_tr = 0
    for para_id, qas in para_qa_train.items():
        for qa in qas:
            if(que_length_tr < len(qas[0].question)):
                que_length_tr = len(qas[0].question)
    
    que_length_dev = 0
    for para_id, qas in para_qa_dev.items():
        for qa in qas:
            if(que_length_dev < len(qas[0].question)):
                que_length_dev = len(qas[0].question)

    if que_length_tr > que_length_dev:
        return que_length_tr
    else:
        return que_length_dev

# Method to pad the contexts
def get_padded_paras(paras, max_length_para):
    paras_padded = dict()
    for key, para in paras.items():
        if len(para) < max_length_para:
            for i in range(max_length_para - len(para)):
                para.append('<PAD>')
        paras_padded[key] = para

    return paras_padded

# Method to give word indices from the character indices for the given context and answer
def getWordIndices(context, answer):
    
    start_cindex = answer['answer_start']
    end_cindex = start_cindex + len(answer['text']) - 1
    curr_cindex = 0

    start_word_index = 0
    end_word_index = 0
    curr_word_index = 0

    for word in context:
        if word in string.punctuation:
            curr_cindex += len(word)
        else:
            curr_cindex += len(word) + 1

        if curr_cindex > start_cindex:
            if(start_word_index == 0):
                start_word_index = curr_word_index

        if(curr_cindex >= end_cindex):
            end_word_index = curr_word_index
            break
        curr_word_index += 1

    return start_word_index, end_word_index
    
# Method to get the words from vocabulary for the respective index
def get_words_from_index(contexts):
      
  word_contexts = list()
  for context in contexts:
    word_context = list()
    for word_index in context:
      word = rev_vocab[word_index]
      word_context.append(word)
    word_contexts.append(word_context)
  return word_contexts


In [0]:
# Method to flatten the normalized structure for the train set
def get_final_cqa(para_qa, paras_padded):
  print('Getting indices for contexts, queries, answers...')
  contexts, queries, answers = list(), list(), list()
  
  for para_id, qas in tqdm(para_qa.items()):
    ctx = paras_padded.get(para_id)
    ctx_indices = get_indices(ctx, vocab)

#     Appending the contexts for each question-answer pair
    for i in range(len(qas)):
        contexts.append(ctx_indices)

    for qa in qas:
        query = qa.question

#         Adding the padding for queries here
        if len(query) < max_length_question:
            for i in range(max_length_question - len(query)):
                query.append('<PAD>')

        queries.append(get_indices(query, vocab))
        answers.append([qa.answers[0].answer_start, qa.answers[0].answer_end])

  return np.asarray(contexts), np.asarray(queries), np.asarray(answers)

# Method to flatten the normalized structure for the dev set
def get_final_cqa_dev(para_qa, paras_padded):
  print('Getting indices for contexts, queries, answers...')
  contexts, queries, answers = list(), list(), list()
  
  for para_id, qas in tqdm(para_qa.items()):
    ctx = paras_padded.get(para_id)
    ctx_indices = get_indices(ctx, vocab)

    context_to_add = contexts.append(ctx_indices)

    for qa in qas:
        query = qa.question

#         Adding the padding for queries here
        if len(query) < max_length_question:
            for i in range(max_length_question - len(query)):
                query.append('<PAD>')

        query_to_add = get_indices(query, vocab)

#         Appending the contexts, queries, answer for each question-answer pair
        for ans in qa.answers:
          answers.append([ans.answer_start, ans.answer_end])
          queries.append(query_to_add)
          contexts.append(context_to_add)

  return np.asarray(contexts), np.asarray(queries), np.asarray(answers)



In [0]:
# A simple baseline model
class Baseline(nn.Module):

  def __init__(self,ctx, query, weight_matrix):
    super(Baseline, self).__init__()
    
    self.embedding = nn.Embedding(weight_matrix.shape[0], weight_matrix.shape[1])
#     self.embedding.load_state_dict({'weight' : weight_matrix})

    self.embedding.weight = nn.Parameter(torch.from_numpy(weight_matrix).type(torch.FloatTensor), requires_grad = True)

    self.embd_length = args.glove_vec_size
    self.contexual_embd = nn.GRU(self.embd_length, self.embd_length, bidirectional=True, batch_first=True)
    self.ctx = ctx
    self.query = query
#     self.dropout = nn.Dropout(p=0.2)

    self.W = nn.Linear(3*self.embd_length,1,bias=False)

    self.modeling_level_1 = nn.GRU(self.query.shape[1],self.embd_length,bidirectional=True,num_layers=1,batch_first=True)
    self.modeling_level_2 = nn.GRU(self.query.shape[1],self.embd_length,bidirectional=True,num_layers=1,batch_first=True)

    self.final_layer = nn.Linear(2*self.embd_length,1)

  def forward(self,ctx,query):
#     sequence lengths for context and query
    batch_size, T, J = ctx.size(0), ctx.size(1), query.size(1)

    ctx = self.embedding(ctx)
    query = self.embedding(query)

#     Create similarity matrix
    
    ctx_sim = ctx.unsqueeze(2).expand((batch_size, T, J, self.embd_length))
    query_sim = query.unsqueeze(1).expand((batch_size, T, J, self.embd_length))
    element_mul = torch.mul(ctx_sim, query_sim)
    concat_ctx_query = torch.cat((ctx_sim, query_sim, element_mul), -1)

#     go from (N,T,J,6*embd_len) to (N,T,J,1) to (N,T,J)
    similarity_matrix = self.W(concat_ctx_query)
    similarity_matrix = similarity_matrix.view(batch_size, T, J)

#     model
    modelled,_ = self.modeling_level_1(similarity_matrix)

    a_begin = self.final_layer(modelled).squeeze()
#     a_begin = self.dropout(a_begin)
#     a_begin = nn.Softmax(dim=-1)(self.final_layer(torch.cat((megamerge,modelled), dim=-1)).squeeze())

    modelled_2,_ = self.modeling_level_2(similarity_matrix)

    a_end = self.final_layer(modelled_2).squeeze()
#     a_end = self.dropout(a_end)
#     a_end = nn.Softmax(dim=-1)(self.final_layer(torch.cat((megamerge,modelled_2), dim=-1)).squeeze())

    return a_begin, a_end



In [0]:
# Defining the Bi-Directional Attention Flow model here
class BIDAF(nn.Module):

  def __init__(self, ctx, query, weight_matrix):
    super(BIDAF, self).__init__()
    
    self.embedding = nn.Embedding(weight_matrix.shape[0], weight_matrix.shape[1])
#     self.embedding.load_state_dict({'weight' : weight_matrix})

    self.embedding.weight = nn.Parameter(torch.from_numpy(weight_matrix).type(torch.FloatTensor), requires_grad = True)

    self.embd_length = args.glove_vec_size
    self.contexual_embd = nn.GRU(self.embd_length, self.embd_length, bidirectional=True, batch_first=True)
    self.ctx = ctx
    self.query = query
#     self.dropout = nn.Dropout(p=0.2)

    self.W = nn.Linear(6*self.embd_length,1,bias=False)

    self.modeling_level_1 = nn.GRU(self.embd_length*8, self.embd_length,bidirectional=True, num_layers=2, batch_first=True)
    self.modeling_level_2 = nn.GRU(self.embd_length*8, self.embd_length,bidirectional=True, num_layers=2, batch_first=True)

    self.final_layer = nn.Linear(10*self.embd_length, 1)

  def forward(self,ctx,query):
#     Sequence lengths for context and query
    batch_size, T, J = ctx.size(0), ctx.size(1), query.size(1)

#     Contexual embeddings for context and query. Remember that these are bidirectional
    ctx = self.embedding(ctx)
    query = self.embedding(query)

    ctx,_ = self.contexual_embd(ctx)
    query,_ = self.contexual_embd(query)

#     Creating similarity matrix
    ctx_sim = ctx.unsqueeze(2).expand((batch_size, T, J, 2*self.embd_length))
    query_sim = query.unsqueeze(1).expand((batch_size, T, J, 2*self.embd_length))
    element_mul = torch.mul(ctx_sim, query_sim)
    concat_ctx_query = torch.cat((ctx_sim, query_sim, element_mul), -1)

#     go from (N,T,J,6*embd_len) to (N,T,J,1) to (N,T,J)
    similarity_matrix = self.W(concat_ctx_query)
    similarity_matrix = similarity_matrix.view(batch_size, T, J)

#     Context-2-Query
    ctx_attention_weights = nn.Softmax(dim=-1)(similarity_matrix)
    c2q = torch.bmm(ctx_attention_weights, query)

#     Query-2-Context
    b = nn.Softmax(dim=-1)(torch.max(similarity_matrix, dim=-1)[0])
    q2c = torch.bmm(b.unsqueeze(1), ctx).repeat(1, T, 1)

#     Merging the contexts, Context2Query, and 
#     itemwise multiplication of contexts with Context2Query and Query2Context
    megamerge = torch.cat((ctx, c2q, torch.mul(ctx, c2q), torch.mul(ctx, q2c)), dim=-1)

#     Passing the merged marix to a GRU to get the starting index of the predicted answer
    modelled,_ = self.modeling_level_1(megamerge)
    a_begin = self.final_layer(torch.cat((megamerge, modelled), dim=-1)).squeeze()
#     a_begin = self.dropout(a_begin)
#     a_begin = nn.Softmax(dim=-1)(self.final_layer(torch.cat((megamerge,modelled), dim=-1)).squeeze())


#     Passing the merged marix to a GRU to get the end index of the predicted answer
    modelled_2,_ = self.modeling_level_2(megamerge)
    a_end = self.final_layer(torch.cat((megamerge, modelled_2), dim=-1)).squeeze()
#     a_end = self.dropout(a_end)
#     a_end = nn.Softmax(dim=-1)(self.final_layer(torch.cat((megamerge,modelled_2), dim=-1)).squeeze())


    return a_begin, a_end


In [0]:
# The Dataset class to store the context, query and answer
class BIDAFDataset(Dataset):
    def __init__(self, ctx, query, answer):
        
        ctx_dtype = torch.cuda.LongTensor
        query_dtype = torch.cuda.LongTensor
        answer_dtype = torch.cuda.LongTensor

        self.length = ctx.shape[0]

        self.ctx = torch.from_numpy(ctx).type(ctx_dtype)
        self.query = torch.from_numpy(query).type(query_dtype)
        self.answer = torch.from_numpy(answer).type(answer_dtype)

    def __getitem__(self, index):
        return self.ctx[index], self.query[index], self.answer[index]

    def __len__(self):
        return self.length

In [0]:
# Method to process the answers 
def process_answers(para, a_pred, answers_test):

# Removing the punctuations if any. 
# This method was used earlier when there were some punctuations along with the words
  def remove_punc(array,a):
    temp =  [word.translate({ord(i):None for i in '!"#$%&\'()*+, -./:;<=>?@[\]^_`{|}~'}) for word in array]
    if len(''.join(temp))==0 and a=='label':
      pass
    return [x for x in temp if x!='']
  a_predictions = []
  a_test = []
  for i,paragraph in enumerate(para):

    paragraph = [rev_vocab[index] for index in paragraph]
    
    prediction = paragraph[a_pred[i,0]:a_pred[i,1]+1]
    a_predictions.append(prediction)
    
    label = paragraph[answers_test[i,0]:answers_test[i,1]+1]
    a_test.append(label)

  return a_predictions, a_test

# Calculating the F1 score
def calc_f1_score(a_predictions, a_test):

  f1_score = 0
  acc_score = 0
  precision,recall = 0,0
  for i in range(len(a_predictions)):

    TP = sum((Counter(a_predictions[i]) & Counter(a_test[i])).values())
    if len(a_predictions[i]) == 0 or (len(a_test[i])) == 0:      
      continue
    precision += TP/(len(a_predictions[i]))
    recall += TP/(len(a_test[i]))    
    
  if precision==0 and recall==0:
    return 0
  assert len(a_predictions)==len(a_test)
  
  f1_score = (2*precision*recall)/(precision+recall)
  f1_score/=len(a_predictions)

  return f1_score

def calc_acc_score(preds,labels):
  return np.sum(np.logical_and(preds[:,0]==labels[:,0],preds[:,1]==labels[:,1]))/labels.shape[0]
    


In [0]:
# Method to predict the answers
def test(model, dataloader):

  model.eval()
  answer_predict_vector, answer_vector, ctx_vector = [], [], []
  for ctx_dl, queries_dl, answers_dl in dataloader:
    
    answer_vector.append(answers_dl.data.cpu().numpy())
    ctx_vector.append(ctx_dl.data.cpu().numpy())
    
    a_begin, a_end = model(ctx_dl, queries_dl)
    predictions_begin = torch.argmax((nn.Softmax(dim=-1)(a_begin)), dim=-1, keepdim=True)
    predictions_end = torch.argmax((nn.Softmax(dim=-1)(a_end)), dim=-1, keepdim=True)
    
    answer_predict_vector.append(torch.cat((predictions_begin, predictions_end),dim=-1).data.cpu().numpy())

  answer_predict_vector = np.concatenate(answer_predict_vector, axis=0)
  answer_vector = np.concatenate(answer_vector, axis=0)
  ctx_vector = np.concatenate(ctx_vector,axis=0)

  return ctx_vector, answer_predict_vector, answer_vector


In [0]:
# Get Preprocessed Data for Dev Set and Train Set
paras_dev, para_qa_dev, paras_train, para_qa_train, vocab, rev_vocab, word2vec, weights_matrix = preprocess_data()

print('No. of Paras in Dev Set: ', len(paras_dev))
print('No. of Paras in Train Set: ', len(paras_train))
print('Vocab Length: ', len(vocab))
print('Vocab word2vec: ', len(word2vec))

# Padding the contexts
max_length_para = get_max_length_paras(paras_train, paras_dev)
paras_train_padded = get_padded_paras(paras_train, max_length_para)
paras_dev_padded = get_padded_paras(paras_dev, max_length_para)

# Padding the queries and getting the flattened structure of contexts, queries and answers
max_length_question = get_max_length_questions(para_qa_train, para_qa_dev)
contexts_tr, queries_tr, answers_tr = get_final_cqa(para_qa_train, paras_train_padded)
contexts_dev, queries_dev, answers_dev = get_final_cqa(para_qa_dev, paras_dev_padded)

# Splitting the train data into train set and validation set
contexts_train, contexts_val, queries_train, queries_val, answers_train, answers_val = train_test_split(contexts_tr, queries_tr, answers_tr, test_size=0.33, random_state=42)

contexts_words_train = get_words_from_index(contexts_train)
contexts_words_val = get_words_from_index(contexts_val)
contexts_words_dev = get_words_from_index(contexts_dev)

# Printing the length of the train, validation and test set
print('Train: Contexts Count:', len(contexts_train))
print('Train: Queries Count:', len(queries_train))
print('Train: Answers Count:', len(answers_train))

print('Val: Contexts Count:', len(contexts_val))
print('Val: Queries Count:', len(queries_val))
print('Val: Answers Count:', len(answers_val))

print('Val: Contexts Count:', len(contexts_dev))
print('Val: Queries Count:', len(queries_dev))
print('Val: Answers Count:', len(answers_dev))


Getting paragraphs, questions and answers...



Creating Vocabulary...
Processing Dev data...



Processing train data...



Vocabulary created!
Now creating word2vec dictionary...
Glove Path:  /content/drive/My Drive/CSE 244/abc/glove.6B/glove.6B.50d.txt
Glove Corpus Size:  400000



Glove word2vec dictionary created!
Generating weight matrix...



Weight matrix generated!
Returning paragraphs, questions and answers...
No. of Paras in Dev Set:  2067
No. of Paras in Train Set:  18896
Vocab Length:  110089
Vocab word2vec:  77307
Getting indices for contexts, queries, answers...



Getting indices for contexts, queries, answers...



Train: Contexts Count: 58691
Train: Queries Count: 58691
Train: Answers Count: 58691
Val: Contexts Count: 28908
Val: Queries Count: 28908
Val: Answers Count: 28908
Val: Contexts Count: 10570
Val: Queries Count: 10570
Val: Answers Count: 10570


In [0]:
# Printing a sample Context, Question and corresponding Answer
print('context: ', paras_dev[1])
print('Question: ', para_qa_dev[1][1].question)
# print(para_qa_dev[0][0].answers)
for ans in para_qa_dev[1][1].answers:
    print('AnsText:', ans.answer_text, ' AnsStart:', ans.answer_start, ' AnsEnd:',ans.answer_end)

context:  ['the', 'panthers', 'finished', 'the', 'regular', 'season', 'with', 'a', '15–1', 'record', ',', 'and', 'quarterback', 'cam', 'newton', 'was', 'named', 'the', 'nfl', 'most', 'valuable', 'player', '(', 'mvp', ')', '.', 'they', 'defeated', 'the', 'arizona', 'cardinals', '49–15', 'in', 'the', 'nfc', 'championship', 'game', 'and', 'advanced', 'to', 'their', 'second', 'super', 'bowl', 'appearance', 'since', 'the', 'franchise', 'was', 'founded', 'in', '1995.', 'the', 'broncos', 'finished', 'the', 'regular', 'season', 'with', 'a', '12–4', 'record', ',', 'and', 'denied', 'the', 'new', 'england', 'patriots', 'a', 'chance', 'to', 'defend', 'their', 'title', 'from', 'super', 'bowl', 'xlix', 'by', 'defeating', 'them', '20–18', 'in', 'the', 'afc', 'championship', 'game', '.', 'they', 'joined', 'the', 'patriots', ',', 'dallas', 'cowboys', ',', 'and', 'pittsburgh', 'steelers', 'as', 'one', 'of', 'four', 'teams', 'that', 'have', 'made', 'eight', 'appearances', 'in', 'the', 'super', 'bowl', '.

In [0]:
# Running the final code from here

# Deciding the length of train and val set
how_much = 2*len(contexts_train)//10
how_much_val = 2*len(contexts_val)//10

# Creating dataset and dataloader objects
dataset_train = BIDAFDataset(contexts_train[:how_much], queries_train[:how_much], answers_train[:how_much])
dataloader_train = DataLoader(dataset_train, args.batch_size, True)

dataset_val = BIDAFDataset(contexts_val[:how_much_val], queries_val[:how_much_val], answers_val[:how_much_val])
dataloader_val = DataLoader(dataset_val, args.batch_size, True)

dataset_dev = BIDAFDataset(contexts_dev, queries_dev, answers_dev)
dataloader_dev = DataLoader(dataset_dev, args.batch_size, True)

model = BIDAF(contexts_train, queries_train, weights_matrix)

#model.load_state_dict(checkpoint['state_dict'])
model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters())
#optimizer.load_state_dict(checkpoint['opt'])

for i in tqdm(range(args.num_epochs)):
    
    model.train()
    loss_e = 0
    loader_count = 0
    for contexts_dl, queries_dl, answers_dl in dataloader_train:

        a_begin, a_end = model.forward(contexts_dl, queries_dl)

        l1 = loss_fn(a_begin, answers_dl[:,0])
        l2 = loss_fn(a_end, answers_dl[:,1])

        optimizer.zero_grad()

        loss = l1+l2
        (loss).backward()

        optimizer.step()
        loss_e += loss.data.item()
        if loader_count%100==0:
          print('loader count:', loader_count)
        loader_count += 1
        
# Printing the loss after each epoch
    print('Epoch: ',i)
    print('Loss:',loss.data.item())

#     Getting the predictions on train and val set to check 
#     the train and validation performance after each epoch
    _ctx_train, _prediction_train, _train = test(model,dataloader_train)
    _ctx_val, _prediction_val, _val = test(model,dataloader_val)

    acc_train = calc_acc_score(_prediction_train,_train)
    acc_val = calc_acc_score(_prediction_val, _val)

    a_prediction_train, a_train = process_answers(_ctx_train,_prediction_train, _train)
    a_prediction_val, a_val = process_answers(_ctx_val, _prediction_val, _val)

    f1_train = calc_f1_score(a_prediction_train,a_train)
    f1_val = calc_f1_score(a_prediction_val,a_val)
    
    print('Train accuracy: ', round(acc_train,5),' Train f1: ', round(f1_train,5),' Val accuracy: ', round(acc_val,5), ' Val f1: ', round(f1_val,5))
    
    torch.save(obj={'state_dict': model.state_dict(), 'opt': optimizer.state_dict()}, f='/content/drive/My Drive/CSE 244/abc/weight/'+str(i)+'_checkpoint.model')


loader count: 0
loader count: 100
Epoch:  0
Loss: 8.762714385986328
Train accuracy:  0.01167  Train f1:  0.10315  Val accuracy:  0.00969  Val f1:  0.09392
seqeval  train:  0.002264464265495855  val:  0.002228266212275126



In [0]:
#Testing
def process_answers_unseen(para,a_pred,queries,answers_test):
  '''
  Takes in the context, predictions, queries and labels (they are all indices and not natural language)
  and returns the natural language for all four (a_test are the labels, everything else is
  as per their names)
  '''
  
  def remove_punc(array,a):
    #e = set(string.punctuation)
    temp =  [word.translate({ord(i):None for i in '!"#$%&\'()*+, -./:;<=>?@[\]^_`{|}~'}) for word in array]
    if len(''.join(temp))==0 and a=='label':
      pass
      #print(temp,array)
    return [x for x in temp if x!='']
  a_predictions = []
  paras = []
  q = []
  a_test = []
  #print(para.shape,'WHOAH')
  #print(len(para), a_pred.shape)
  for i,paragraph in enumerate(para):
    #print(paragraph, paragraph[a_begin[i]],paragraph[a_end[i]])
    #print(paragraph.shape)
    paragraph = [rev_vocab[index] for index in paragraph]
    #print(paragraph)
    prediction = paragraph[a_pred[i,0]:a_pred[i,1]+1]
    label = paragraph[answers_test[i,0]:answers_test[i,1]+1]
    #if len(label)==1:
      #print('l',label,answers_test[i,0],answers_test[i,1],rev_vocab[answers_test[i,0]],rev_vocab[answers_test[i,1]])
    #print('l',label)
    query = [rev_vocab[index] for index in queries[i]]

    #prediction,label = remove_space(prediction),remove_space(label)
    #prediction,label = remove_punc(prediction,'prediction'),remove_punc(label,'label')



    a_predictions.append(' '.join(prediction))
    paras.append(' '.join(paragraph))
    q.append(' '.join(query))
    a_test.append(' '.join(label))






  
  return a_predictions,paras,q,a_test



def test_2(model, dataloader):
  '''
  Similar to test except it also return the context vector
  '''
  model.eval()
  answer_predict_vector, answer_vector,ctx_vector,q_vector = [],[],[],[]
  for ctx_dl ,queries_dl, answers_dl in dataloader:
    a_begin,a_end = model(ctx_dl, queries_dl)
    answer_vector.append(answers_dl.data.cpu().numpy())
    ctx_vector.append(ctx_dl.data.cpu().numpy())
    q_vector.append(queries_dl.data.cpu().numpy())
    predictions_begin = torch.argmax((nn.Softmax(dim=-1)(a_begin)), dim=-1, keepdim=True)
    predictions_end = torch.argmax((nn.Softmax(dim=-1)(a_end)), dim=-1, keepdim=True)
    answer_predict_vector.append(torch.cat((predictions_begin,predictions_end),dim=-1).data.cpu().numpy())

  answer_predict_vector = np.concatenate(answer_predict_vector, axis=0)
  answer_vector = np.concatenate(answer_vector, axis=0)
  ctx_vector = np.concatenate(ctx_vector,axis=0)
  q_vector = np.concatenate(q_vector,axis=0)

  return ctx_vector, answer_predict_vector, q_vector, answer_vector

def test_unseen(model,test_loader):

  ctx_vector,answer_prediction_vector,q_vector,answer_vector = test_2(model,dataloader_dev)
  answers,paras,q,actual_answers = process_answers_unseen(ctx_vector,answer_prediction_vector,q_vector,answer_vector)
  final = pd.DataFrame({'Paras':paras,'Q':q,'Prediction':answers, 'Answers':actual_answers})

  return final 

In [0]:
final = test_unseen(model,dataloader_dev)

In [0]:
i=11
final['Paras'][i]

"on 7 january 1943 , at the age of 86 , tesla died alone in room 3327 of the new yorker hotel . his body was later found by maid alice monaghan after she had entered tesla 's room , ignoring the `` do not disturb '' sign that tesla had placed on his door two days earlier . assistant medical examiner h.w . wembly examined the body and ruled that the cause of death had been coronary thrombosis . tesla 's remains were taken to the frank e. campbell funeral home at madison ave. and 81st st. a long-time friend and supporter of tesla , hugo gernsback , commissioned a sculptor to create a death mask , now displayed in the nikola tesla museum . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <

In [0]:
final['Q'][i]

"what was the date of tesla 's death ? <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [0]:
final['Prediction'][i]

'tesla died alone in room 3327 of the new yorker hotel'

In [0]:
final['Answers'][i]

'7 january 1943'